# Webscraping

---

## Scrapen van een website met Selenium

Scrapen met [Selenium] is niet de meest betrouwbare manier van webscraping.  
Maar sommige websites hebben (veel) Javascript of [iframe] HTML tags of laden dynamisch nieuwe elementen waardoor de data moeilijk te scrapen is met andere technieken.  
Het is dan toch mogenlijk om de website te scrapen met Selenium.

---


https://sroze.github.io/ngInfiniteScroll/demo_async.html is een demo website voor een [infinite scroll] Javascript module die werkt met het webapplicatieframework [AngularJS].  
De website gebruikt de [Reddit API] om tot en met 1000 artikelen te laden als de einde van de pagina wordt bereikt.  
In de notebook cells hieronder open we de demo website en gebruiken we Selenium (met een beetje Javascript) om te scrollen.
De artikelen parsen 



[iframe]: https://www.w3docs.com/learn-html/html-iframe-tag.html
[Selenium]: https://pypi.org/project/selenium/
[webdriver_manager]: https://pypi.org/project/webdriver-manager/ 

[infinite scroll]: https://github.com/sroze/ngInfiniteScroll
[AngularJS]: https://angularjs.org/

[Reddit API]: https://www.reddit.com/dev/api


In [1]:
from collections import namedtuple

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.remote.webelement import WebElement

from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType

# driver = webdriver.Chrome(ChromeDriverManager().install())  # chrome
driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install())  # chromium
# driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())  # firefox



====== WebDriver manager ======
Current chromium version is 94.0.4606
Get LATEST driver version for 94.0.4606
There is no [linux64] chromedriver for browser 94.0.4606 in cache
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_linux64.zip
Driver has been saved in cache [/home/egraaf/.wdm/drivers/chromedriver/linux64/94.0.4606.61]


WebDriverException: Message: unknown error: DevToolsActivePort file doesn't exist


Er browser die geopend is bij het aanroepen van de `webdriver.Chrome`.   
Deze browser wordt nu gecommandeerd door Python via het `driver` object.  

Selenium heeft geen commando om de browser te scrollen maar Javascript wel.  
Selenium kan javascript injecteren in de browser die gestart is door Selenium.

Hieronder een functie die de viewport van de browser naar beneden laat scrollen.  

In [ ]:
def scroll_to_end(driver: WebDriver):
    """use javascript to scroll to the bottom of the website"""
    code = 'window.scrollTo(0, document.body.scrollHeight)'
    driver.execute_script(code)

Door een URL te geven aan het `get` functie van het `webdriver` object wordt er een webpagina geopend in de browser.  

In [ ]:
url = 'https://sroze.github.io/ngInfiniteScroll/demo_async.html'
driver.get(url)

In de developer tools onder de tab Elements kan er een [XPath] worden gemaakt om een pad naar `WebElement` te maken.  
De input om dit te doen wordt met de toetsencombinatie `Ctrl + F` geactiveerd. 

Met de functie `find_elements_by_xpath` _(elements meervoud)_ wordt alle webelementen opgevraagd die dat XPath hebben.

[XPath]: https://nl.wikipedia.org/wiki/XPath

In [ ]:
xpath_item = "//div[@class='demo-container']/div[contains(@class, 'item')]"
items = driver.find_elements_by_xpath(xpath_item)
items[0]  # check het eerste object in de lijst met WebElements

Hieronder is er een functie gemaakt die de `WebElement` als argument verwacht.  
Met BeautifulSoup wordt dan de HTML geparsed om alle data te verkrijgen.  
De data wordt opgevraagd, opgeschoond en direct in een `dict` geplaatst.  
De `dict` met data wordt na het voltooien van de functie teruggegeven door de functie.  

In [ ]:
def get_data(item: WebElement) -> dict:
    assert isinstance(item, WebElement), f'received type: {type(item)!r}'    
    html = item.get_attribute('outerHTML')
    soup = BeautifulSoup(html, 'html.parser')
    result = {
        'score': int(soup.find('span', {'class': 'score'}).text.strip()),  # string to int
        'title': soup.find('span', {'class': 'title'}).text.strip(),
        'url':   soup.find('a').get('href')
    }
    return result

Hieronder de geparste data van het eerste object in de lijst met WebElements.  

In [ ]:
get_data(items[0])

Alle logica is gecreëerd om de browser te scrollen, data op te vragen en de data te parsen.  
Om alle data op te vragen wordt hieronder een `while` loop gebruikt.  

In [10]:
items = []
while len(items) < 999:
    items = driver.find_elements_by_xpath(xpath_item)
    scroll_to_end(driver)    

In [15]:
len(items)  # check hoe veel objecten de lijst items bevat

1000

Alle data is verkregen en nu kan hier wat mee worden gedaan.  
Bijvoorbeeld kan het artikel met de hoogste score worden opgevraagd.  

In [21]:
# een generator is een luie functie die data geeft waneer dit wordt opgevraagd.  
item_generator = (get_data(item) for item in items)

# verklaar objecten die gebruikt worden in de for-loop
best_article = {}
max_score = 0

# loop om het artikel met de hoogste score te verkrijgen
for data in item_generator:
    score = data['score']
    if max_score < score:
        max_score = score
        best_article = data

best_article

{'score': 103407,
 'title': 'The best come back ever',
 'url': 'https://v.redd.it/jg0krsmkj4p71'}

In [12]:
driver.quit()


NameError: name 'driver' is not defined